In [1]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Import the optimise sublibrary
import scipy.optimize as sco
# Import cubic splines interpolation module
import scipy.interpolate as sci

In [2]:

msft = yf.Ticker("MSFT")
aapl_historical = msft.history(start="2020-06-02", end="2020-07-07")

aapl_historical
data = yf.download("AMZN AAPL GOOG ^GSPC",  start="2017-01-01", end="2018-01-01")
def df(data,pm):
    df=data[pm]
    return df
data=df(data,"Close")
data

[*********************100%***********************]  4 of 4 completed


,AAPL,AMZN,GOOG,^GSPC
Date,,,,
2017-01-03,29.037500,37.683498,39.306999,2257.830078
2017-01-04,29.004999,37.859001,39.345001,2270.750000
2017-01-05,29.152500,39.022499,39.701000,2269.000000
2017-01-06,29.477501,39.799500,40.307499,2276.979980
2017-01-09,29.747499,39.846001,40.332500,2268.899902
...,...,...,...,...
2017-12-22,43.752499,58.417999,53.006001,2683.340088
2017-12-26,42.642502,58.838001,52.837002,2680.500000
2017-12-27,42.650002,59.112999,52.468498,2682.620117


In [3]:
# Normalize stock data based on initial price
def normalize(data):
  x = data.copy()
  for i in x.columns[0:]:
    x[i] = x[i]/x[i][0]
  return x
normalize(data)

# Function to plot interactive plot
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[0:]:
    fig.add_scatter(x = df.index, y = df[i], name = i)
  fig.show()


In [4]:
interactive_plot(normalize(data), 'Normalized Prices')

In [5]:
def daily_return(df):
    df_daily_return = df.copy()
    df_daily_return=df_daily_return.pct_change()
    for i in df.columns[0:].tolist():
        df_daily_return[i][0] = 0
    return df_daily_return
stocks_daily_return = daily_return(data).iloc[:,:(len(data.columns)-1)]
stocks_daily_return


,AAPL,AMZN,GOOG
Date,,,
2017-01-03,0.000000,0.000000,0.000000
2017-01-04,-0.001119,0.004657,0.000967
2017-01-05,0.005085,0.030732,0.009048
2017-01-06,0.011148,0.019912,0.015277
2017-01-09,0.009159,0.001168,0.000620
...,...,...,...
2017-12-22,0.000000,-0.005448,-0.003300
2017-12-26,-0.025370,0.007190,-0.003188
2017-12-27,0.000176,0.004674,-0.006974


In [6]:
def generate_portfolios(df,n):
    rs=[]
    sts=[]
    for i in range(n):
        weights=np.random.random(len(df.columns[:(len(stocks_daily_return.columns))]))
        weights /= np.sum(weights)
        rs.append(np.sum(df.iloc[:,:(len(stocks_daily_return.columns))].mean()*weights)*252)
        sts.append(np.sqrt(np.dot(weights.T,np.dot(df.iloc[:,:(len(stocks_daily_return.columns))].cov()*252,weights))))
    rs=np.array(rs)
    sts=np.array(sts)
    return rs,sts,weights

In [7]:
a=generate_portfolios(stocks_daily_return,5000)
risk_free_rate=0
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [8]:
data

,AAPL,AMZN,GOOG,^GSPC
Date,,,,
2017-01-03,29.037500,37.683498,39.306999,2257.830078
2017-01-04,29.004999,37.859001,39.345001,2270.750000
2017-01-05,29.152500,39.022499,39.701000,2269.000000
2017-01-06,29.477501,39.799500,40.307499,2276.979980
2017-01-09,29.747499,39.846001,40.332500,2268.899902
...,...,...,...,...
2017-12-22,43.752499,58.417999,53.006001,2683.340088
2017-12-26,42.642502,58.838001,52.837002,2680.500000
2017-12-27,42.650002,59.112999,52.468498,2682.620117


In [26]:
stocks_daily_return

,AAPL,AMZN,GOOG
Date,,,
2017-01-03,0.000000,0.000000,0.000000
2017-01-04,-0.001119,0.004657,0.000967
2017-01-05,0.005085,0.030732,0.009048
2017-01-06,0.011148,0.019912,0.015277
2017-01-09,0.009159,0.001168,0.000620
...,...,...,...
2017-12-22,0.000000,-0.005448,-0.003300
2017-12-26,-0.025370,0.007190,-0.003188
2017-12-27,0.000176,0.004674,-0.006974


: 

In [9]:
def ptf_stats(weights):
    weights = np.array(weights)
    ptf_r = np.sum(stocks_daily_return.mean() * weights) * 252
    ptf_std = np.sqrt(np.dot(weights.T, np.dot(stocks_daily_return.iloc[:,:(len(stocks_daily_return.columns))].cov() * 252, weights)))
    return {"return":ptf_r,"volatility": ptf_std, "sharpe":(ptf_r) / ptf_std}

# Minimise the negative value of the Sharpe ratio
def min_sharpe(weights):
    return -ptf_stats(weights)["sharpe"]
    # Write the constraint that the weights have to add up to 1

def minimize_volatility(weights):  
    # Note that we don't return the negative of volatility here because we 
    # want the absolute value of volatility to shrink, unlike sharpe.
    return ptf_stats(weights)["volatility"] 
def minimize_return(weights): 
    return -ptf_stats(weights)["return"]
    
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Bound the weights (parameter inputs) to be within 0 and 1
bnds = tuple((0, 1) for x in range((len(data.columns)-1)))


# Starting parameter (weights) list as equal distribution
starting_ws = (len(data.columns)-1) * [1. / (len(data.columns)-1),]

# Call the minimisation function
opts = sco.minimize(min_sharpe, starting_ws, method='SLSQP', bounds=bnds, constraints=cons)
# Inspect the results




In [10]:
optimal_sharpe_weights=opts['x'].round(4)
list(zip(data.columns[:len(stocks_daily_return)],list(optimal_sharpe_weights)))

[('AAPL', 0.4745), ('AMZN', 0.3387), ('GOOG', 0.1867)]

In [11]:
# Obtain the optimal weights
weights_opt = opts['x'].round(3)
weights_opt

# Plug optimal weights into the statistics function



array([0.475, 0.339, 0.187])

In [12]:

# Plug optimal weights into the statistics function
ptf_stats(weights_opt)

{'return': 0.3997557969049422,
 'volatility': 0.15531510250876232,
 'sharpe': 2.5738372537364125}

In [13]:
optimal_stats = ptf_stats(optimal_sharpe_weights)
print(optimal_stats)

print('Optimal Portfolio Return: ', round(optimal_stats["return"]*100,4))
print('Optimal Portfolio Volatility: ', round(optimal_stats["volatility"]*100,4))
print('Optimal Portfolio Sharpe Ratio: ', round(optimal_stats["sharpe"],4))

{'return': 0.3993304569976965, 'volatility': 0.15514984718004765, 'sharpe': 2.5738372564059513}
Optimal Portfolio Return:  39.933
Optimal Portfolio Volatility:  15.515
Optimal Portfolio Sharpe Ratio:  2.5738


In [14]:



# Define a function that minimises portfolio variance
def min_var(weights):
    # Remember that variance is just standard deviation (volatility) squared
    return ptf_stats(weights)["volatility"]
# Call the optimisation fcuntion
opt_var = sco.minimize(min_var, starting_ws, method='SLSQP', bounds=bnds, constraints=cons)
# Inspect the results
opt_var



     fun: 0.14091612622866545
     jac: array([0.14090335, 0.14609294, 0.14092345])
 message: 'Optimization terminated successfully'
    nfev: 28
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([3.64202685e-01, 3.46944695e-18, 6.35797315e-01])

In [15]:
# Obtain the optimal weights
weights_opt_var = opt_var['x'].round(3)
weights_opt_var


array([0.364, 0.   , 0.636])

In [16]:

# Get the statistics for the absolute minimum variance portfolio
ptf_stats(weights_opt_var)

{'return': 0.33346026638243986,
 'volatility': 0.14091613440965026,
 'sharpe': 2.366373927154815}

In [17]:


# Set up two conditions, one for the target return level and one for the sum of the portfolio weights
cons2 = ({'type': 'eq', 'fun': lambda x: ptf_stats(x)["return"] - r},
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# The boundary condition stays the same
bnds2 = tuple((0, 1) for x in weights_opt)
# Define a function that returns the volatility of a portfolio given a vector of weights
def min_port(weights):
    return ptf_stats(weights)["volatility"]

In [18]:
# Define a function to get the target returns and volatilities given a range of returns
def efficient_frontier(start_r, end_r, steps):
    target_rs = np.linspace(start_r, end_r, steps)
    target_stds = []
    for r in target_rs:
        cons2 = ({'type': 'eq', 'fun': lambda x: ptf_stats(x)["return"] - r},
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bnds2 = tuple((0, 1) for x in a[2])
        opt_var = sco.minimize(min_var, starting_ws, method='SLSQP', bounds=bnds2, constraints=cons2)
        target_stds.append(opt_var['fun'])
    target_stds = np.array(target_stds)
    return target_rs, target_stds

target_rs, target_stds= efficient_frontier(a[0].min(),a[0].max(),50)

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, marker=dict(symbol="x",color=((target_rs - 0.01)/target_stds), 
                                  showscale=False, 
                                  size=7,colorscale="RdBu"
                                  ),mode="markers"
)

)
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=[ptf_stats(weights_opt_var)['volatility']],y=[ptf_stats(weights_opt_var)['return']],marker=dict(symbol="x",size=10),mode="markers"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [22]:

# Select the index of the absolute minimum variance portfolio
min_var = np.argmin(target_stds)

# Select expected volatilities for the stocks on the efficient frontier
ex_stds = target_stds[min_var:]

# Select expected returns for the stocks on the efficient frontier
ex_rs = target_rs[min_var:]

# Interpolate the B-spline representation of the data points
tck = sci.splrep(ex_stds, ex_rs)

In [23]:
# Define a continuously differentiable function f(x) for the efficient frontier
def f(x):
    return sci.splev(x, tck, der=0)

# Define te first derivate function of f(x)
def df(x):
    return sci.splev(x, tck, der=1)

# Set up a system of equations for the conditions that the CML has to satisfy
def cml_conditions(p, rf=0.01):
    eq1 = rf - p[0]
    eq2 = rf + p[1] * p[2] - f(p[2])
    eq3 = p[1] - df(p[2])
    return eq1, eq2, eq3

# Solve for the parameters of the Capital Market Line
cml = sco.fsolve(cml_conditions, [0, 0.5, 0.15])
# Create input variable for CML
cx = np.linspace(a[1].min(),a[1].max())

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a[1], 
                         y=a[0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(a[1]-risk_free_rate)/(a[1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=cx,y=(cml[0] + cml[1] * cx),mode="lines"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=target_stds, y=target_rs, mode="lines"
)

)
fig.add_trace(go.Scatter(x=[ptf_stats(optimal_sharpe_weights)['volatility']],y=[ptf_stats(optimal_sharpe_weights)['return']],marker=dict(symbol="x",size=10),mode="markers"))
fig.add_trace(go.Scatter(x=cx,y=(cml[0] + cml[1] * cx),mode="lines"))

#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))